In [1]:
!python -V

Python 3.13.2


# Run MLFlow service with Sqlite backend

In [ ]:
!mlflow ui --backend-store-uri sqlite:///mlflow.db

SyntaxError: invalid syntax (2109785657.py, line 1)

# Import and Setup MLFlow library

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlops-experiment")

<Experiment: artifact_location='/Users/sstamkulov/Documents/ml/jupyter_notebooks_archive/datatalks_ml_zoomcamp/module_2_mlflow/mlruns/1', creation_time=1748153971145, experiment_id='1', last_update_time=1748153971145, lifecycle_stage='active', name='mlops-experiment', tags={}>

# Train and Evaluate model

In [4]:
import pandas as pd

In [5]:
import pickle

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [27]:
train_file = '../data/green_tripdata_2025-01.parquet'
val_file = '../data/green_tripdata_2025-02.parquet'

df_train = read_dataframe(train_file)
df_val = read_dataframe(val_file)

In [10]:
len(df_train), len(df_val)

(46307, 44218)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

6.098871741054109

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", train_file)
    mlflow.log_param("valid-data-path", val_file)

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [20]:
import xgboost as xgb

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:7.36228                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [08:29:49] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:6.42930                           
[2]	validation-rmse:5.96119                           
[3]	validation-rmse:5.72710                           
[4]	validation-rmse:5.60752                           
[5]	validation-rmse:5.54008                           
[6]	validation-rmse:5.51165                           
[7]	validation-rmse:5.49392                           
[8]	validation-rmse:5.48451                           
[9]	validation-rmse:5.47454                           
[10]	validation-rmse:5.46993                          
[11]	validation-rmse:5.46716                          
[12]	validation-rmse:5.46599                          
[13]	validation-rmse:5.46557                          
[14]	validation-rmse:5.46580                          
[15]	validation-rmse:5.46457                          
[16]	validation-rmse:5.46440                          
[17]	validation-rmse:5.46204                          
[18]	validation-rmse:5.46064                          
[19]	valid

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [08:29:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:7.67639                                                    
[2]	validation-rmse:7.16716                                                    
[3]	validation-rmse:6.76328                                                    
[4]	validation-rmse:6.44801                                                    
[5]	validation-rmse:6.20255                                                    
[6]	validation-rmse:6.01144                                                    
[7]	validation-rmse:5.86590                                                    
[8]	validation-rmse:5.75407                                                    
[9]	validation-rmse:5.66970                                                    
[10]	validation-rmse:5.60485                                                   
[11]	validation-rmse:5.55757                                                   
[12]	validation-rmse:5.52099                                                   
[13]	validation-rmse:5.49167            

...

[1]	validation-rmse:7.36629                                                     
[2]	validation-rmse:6.81865                                                     
[3]	validation-rmse:6.42193                                                     
[4]	validation-rmse:6.13731                                                     
[5]	validation-rmse:5.93441                                                     
[6]	validation-rmse:5.79336                                                     
[7]	validation-rmse:5.69625                                                     
[8]	validation-rmse:5.62531                                                     
[9]	validation-rmse:5.57595                                                     
[10]	validation-rmse:5.54167                                                    
[11]	validation-rmse:5.51563                                                    
[12]	validation-rmse:5.49770                                                    
[13]	validation-rmse:5.48271

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [08:36:25] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[3]	validation-rmse:5.75782                                                     
[4]	validation-rmse:5.63784                                                     
[5]	validation-rmse:5.57783                                                     
[6]	validation-rmse:5.52729                                                     
[7]	validation-rmse:5.50883                                                     
[8]	validation-rmse:5.49260                                                     
[9]	validation-rmse:5.48536                                                     
[10]	validation-rmse:5.48358                                                    
[11]	validation-rmse:5.48439                                                    
[12]	validation-rmse:5.48563                                                    
[13]	validation-rmse:5.48574                                                    
[14]	validation-rmse:5.48086                                                    
[15]	validation-rmse:5.47995

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [08:36:29] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[3]	validation-rmse:5.58324                                                     
[4]	validation-rmse:5.54042                                                     
[5]	validation-rmse:5.51767                                                     
[6]	validation-rmse:5.50187                                                     
[7]	validation-rmse:5.47968                                                     
[8]	validation-rmse:5.47584                                                     
[9]	validation-rmse:5.46958                                                     
[10]	validation-rmse:5.46551                                                    
[11]	validation-rmse:5.46196                                                    
[12]	validation-rmse:5.45890                                                    
[13]	validation-rmse:5.45689                                                    
[14]	validation-rmse:5.45510                                                    
[15]	validation-rmse:5.45343

In [28]:
mlflow.xgboost.autolog(disable=True)

In [30]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:8.56745


/opt/anaconda3/envs/ml/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [08:37:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[1]	validation-rmse:8.10033
[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059
[10]	validation-rmse:6.00193
[11]	validation-rmse:5.90700
[12]	validation-rmse:5.82752
[13]	validation-rmse:5.76123
[14]	validation-rmse:5.70306
[15]	validation-rmse:5.65560
[16]	validation-rmse:5.61619
[17]	validation-rmse:5.58065
[18]	validation-rmse:5.55157
[19]	validation-rmse:5.52734
[20]	validation-rmse:5.50858
[21]	validation-rmse:5.48872
[22]	validation-rmse:5.47379
[23]	validation-rmse:5.45957
[24]	validation-rmse:5.44705
[25]	validation-rmse:5.43706
[26]	validation-rmse:5.42713
[27]	validation-rmse:5.41891
[28]	validation-rmse:5.41283
[29]	validation-rmse:5.40819
[30]	validation-rmse:5.40244
[31]	validation-rmse:5.39854
[32]	validation-rmse:5.39446
[33]	validation-rmse:5.39004
[34]	validation-rmse:5.38685
[35]	validation-rmse:5.

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [08:37:42] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/25 08:37:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [32]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

TypeError: got an unexpected keyword argument 'squared'